Importação dos pacotes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression 
from app import func as f

path_arquivo="" #Adicionar o caminho do arquivo a ser consumido, incluindo o nome do mesmo

df=pd.read_csv(path_arquivo, encoding="utf-16", header=0, delim_whitespace=True, index_col=0)
df['Dif']=df['Baseline(µV)']-df['HeatFlow(µV)']

df

Regressão Linear para obter taxa de aquecimento


In [ ]:
model = LinearRegression()
x=np.array(df["Time(s)"]).reshape(-1,1)
y=np.array(df["Furnace_Temperature(°C)"])

model.fit(x,y)
m=model.coef_[0]
b=model.intercept_

with open("Outputs/Dados Coletados - Chumbo.txt", "w", encoding='utf-8') as arquivo:
    arquivo.write("Taxa de aquecimento: %f\nCoeficiente Linear: %f\n" %(m,b))

df["Derivada(Txt)"]=f.derivada(df["Time(s)"],df["Furnace_Temperature(°C)"])


Temperatura de pico

In [ ]:
pico=df["Dif"].max()

indice_pico=df.index[df["Dif"]==pico]

Tpico=df.loc[indice_pico]["Sample_Temperature(°C)"].values[0]


Plot Fluxo por Temperatura e Derivada

In [ ]:
#Variação do fluxo em relação à temperatura da amostra
fluxoportemperatura=f.derivada(df["Sample_Temperature(°C)"],df["HeatFlow(µV)"])

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Temperatura da Amostra (°C)')
ax1.set_ylabel('Fluxo de calor (µV)', color=color)
ax1.plot(df["Sample_Temperature(°C)"],df["HeatFlow(µV)"], label="Dados do Aquecimento",color=color)
ax1.tick_params(axis='y', labelcolor = color)

ax2=ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Derivada (µV/°C)', color=color)

#limite inferior adicionado para eliminar o ruído no início do gráfico
ax2.plot(df[df["Time(s)"]>200]["Sample_Temperature(°C)"],fluxoportemperatura[201:],color=color)
ax2.tick_params(axis='y', labelcolor = color)

fig.tight_layout()

fig=plt.gcf()

plt.show()

fig.savefig("Outputs/Fluxo de calor por Temperatura")
plt.close(fig)

Encontrando máximo e mínimo da derivada (pontos de inflexão) e reta tangente

In [ ]:
df["Derivada(Flux,TempAmos)"]=fluxoportemperatura

#ponto máximo e mínimo da variação
maximo=df[df["Time(s)"]>200]["Derivada(Flux,TempAmos)"].max()
minimo=df[df["Time(s)"]>200]["Derivada(Flux,TempAmos)"].min()

#ponto de inflexão onset (no gráfico, é visível que o onset é um ponto de mínimo)
T_inf1=df[df["Derivada(Flux,TempAmos)"]==minimo]["Sample_Temperature(°C)"].values[0]
F_inf1=df[df["Derivada(Flux,TempAmos)"]==minimo]["HeatFlow(µV)"].values[0]

#ponto de inflexão endset (no gráfico, é visível que o onset é um ponto de máximo)
T_inf2=df[df["Derivada(Flux,TempAmos)"]==maximo]["Sample_Temperature(°C)"].values[0]
F_inf2=df[df["Derivada(Flux,TempAmos)"]==maximo]["HeatFlow(µV)"].values[0]

c_inf1=F_inf1-(minimo*T_inf1) #coeficiente linear da reta tangente de onset
c_inf2=F_inf2-(maximo*T_inf2) #coeficiente linear da reta tangente de endset

Encontrar a linha de base pré e pós evento

In [ ]:
#Achar a linha de base a partir das inflexões das derivadas
indice_inflex_1=df.index[df["Sample_Temperature(°C)"]==T_inf1]                                                  #indice do ponto de inflexão para limitar o intervalo de busca
T_base_set=f.linhabaseonset(df["Derivada(Flux,TempAmos)"],df["Sample_Temperature(°C)"],indice_inflex_1)[0]        #Pelo indice, encontra ao redor a ultima temperatura na linha de base 
indice_base_set=f.linhabaseonset(df["Derivada(Flux,TempAmos)"],df["Sample_Temperature(°C)"],indice_inflex_1)[1]        #e seu valor de fluxo

#Tendset e seu indice para encontrar o Fluxo de Endset
indice_inflex_2=df.index[df["Sample_Temperature(°C)"]==T_inf2]                                                  #indice do ponto de inflexão para limitar o intervalo de busca
T_base_endset=f.linhabaseendset(df["Derivada(Flux,TempAmos)"],df["Sample_Temperature(°C)"],indice_inflex_2)[0]    #Pelo indice, encontra ao redor a primeira temperatura na linha de base 
indice_base_endset=f.linhabaseendset(df["Derivada(Flux,TempAmos)"],df["Sample_Temperature(°C)"],indice_inflex_2)[1]       #e seu valor de fluxo

Tinicio, Tonset, Tendset, Tfinal e Tpico

In [ ]:
#As temperaturas de início (Tonset) e fim (Tendset) são justamente os pontos das linhas de base.
#Consideramos como temperatura de onset e endset os pontos de interseção entre as linhas de base e as retas tangentes nos pontos de inflexão

Intersec_set=df.loc[indice_base_set]["HeatFlow(µV)"].values[0]
Intersec_end=df.loc[indice_base_endset]["HeatFlow(µV)"].values[0]

#Aqui encontramos a temperatura em que as linhas de base cruzam as retas tangentes aos pontos de inflexão
Tonset=(Intersec_set-c_inf1)/minimo
Tendset=(Intersec_set-c_inf2)/maximo

with open("Outputs/Dados Coletados - Chumbo.txt", "a", encoding='utf-8') as arquivo:
    arquivo.write("Temperatura de início: %f °C"%T_base_set.values[0])
    arquivo.write("\nTemperatura de onset: %f °C" %Tonset)
    arquivo.write("\nTemperatura de pico: %f °C" %Tpico)
    arquivo.write("\nTemperatura de endset: %f °C" %Tendset)
    arquivo.write("\nTemperatura final: %f °C\n"%T_base_endset.values[0])

DSC x Tempo

In [ ]:
plt.plot(df["Time(s)"],df["Dif"], label="Dados do Aquecimento",color="red")
plt.xlabel("Tempo (s)")
plt.ylabel("Diferença de fluxo (µV)")
plt.title("Diferença de fluxo entre amostra e referência em função da temperatura")

fig=plt.gcf()

plt.show()

fig.savefig("Outputs/Diferença de Fluxo entre a baseline e a amostra")
plt.close(fig)


Cálculo da Entalpia

In [ ]:
area=sum(f.integral(df["Time(s)"],df["Dif"]))

#Parâmetros a serem corrigidos a depender da calibração do equipamento e da amostra
K=248

m=30.2*10**(-3)

H=area/(m*K)

with open("Outputs/Dados Coletados - Chumbo.txt", "a", encoding='utf-8') as arquivo:
    arquivo.write("Entalpia da reação: %f J/g\n"%H)